In [155]:
import tensorflow

In [156]:
import keras

In [157]:
import librosa

# Test librosa and load a sample file

In [10]:
 y, sr = librosa.load("ToyDatasetSkim//blues_skim.00000.au",
                      duration=2.0)

In [11]:
y.size

44100

In [12]:
# librosa.output.write_wav('file_trim_2s.wav', y, sr)

# Gaussian noise

In [13]:
len(y)
import numpy as np
noise = np.float32(np.random.normal(0,0.04,y.shape))

In [14]:
inputSize = 128

# Add noise to dataset and create toydataset

In [15]:
i = 0
import os
cleanSignal = []
noisySignal = []
for filename in os.listdir("ToyDataset"):
     y, sr = librosa.load("ToyDataset\\" + filename, duration=2.0)
     y1 = y + noise
#      print(y.shape)
     cleanSignal.append(y[0:inputSize])
     noisySignal.append(y1[0:inputSize])
cleanSignal = np.array(cleanSignal)
noisySignal = np.array(noisySignal)

print(cleanSignal.shape)
print(noisySignal.shape)


(93, 128)
(93, 128)


In [ ]:
# i = 0
# import os
# for filename in os.listdir("ToyDataset"):
#      y, sr = librosa.load("ToyDataset\\" + filename, duration=2.0)
#      files = filename.split('.')
#      newfilename = files[0] + '_skim' + '.' + files[1] + "." + files[2]
#      noisefilename = files[0] + '_skimnoisy' + '.' + files[1] + "." + files[2]
#      y1 = y + noise
#      #print(newfilename)
#      librosa.output.write_wav("ToyDatasetSkim\\" + newfilename, y, sr)
#      librosa.output.write_wav("ToyDatasetNoise\\" + noisefilename, y1, sr)
    
    

In [16]:

y


array([-0.00131226, -0.00311279, -0.00738525, ...,  0.07067871,
        0.06311035,  0.05014038], dtype=float32)

In [17]:
len(y)

44100

In [18]:
y1 = y + noise
y1


array([ 0.03811701, -0.0542963 , -0.04267868, ...,  0.00347507,
        0.04576261,  0.03206605], dtype=float32)

In [19]:
# librosa.output.write_wav('file_trim_2s_noise.wav', y1, sr)

# Let's train

In [20]:
import numpy as np
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import SGD,Adam,RMSprop
from keras.models import Model
from keras.layers import Input, merge
from keras.layers import Conv1D,Conv2D,Conv2DTranspose,Reshape
from scipy import io
from keras.layers import LeakyReLU
from keras.layers.advanced_activations import PReLU
from keras.layers import BatchNormalization
from keras import backend as K  
import matplotlib.pyplot as plt

In [21]:
def make_trainable(model, trainable):
    model.trainable = trainable
    for l in model.layers:
        l.trainable = trainable

In [22]:
def D_loss(y_true,y_pred):
    return 0.5*K.mean((y_pred-y_true)**2,axis = -1)

In [23]:
def G_loss(fake_output,true_input):
    def lossfun(y_true, y_pred):
        return 1*K.mean(K.abs(fake_output-true_input)) 
    return lossfun

In [24]:
def GAN_loss(fake_output,true_input):
    def lossfun(y_true, y_pred):
        return 0.5*K.mean((((y_pred-y_true)**2)),axis = -1)+100*K.mean(K.abs(fake_output-true_input))
    return lossfun

In [25]:
optim =RMSprop(lr=0.0002)

# Generator Model 

In [26]:
#input is noise and input1 is clean
inputs=Input(shape =(inputSize,1,1))
inputs1=Input(shape =(inputSize,1))

In [27]:
#combine noise and input
input=([inputs,inputs1])

## Encoder is starting here

In [28]:
#64 kernels of size 31 and strides of 4
cov1=(Conv2D(64, 31, strides = 4,padding='same'))(inputs)
cov1=(PReLU())(cov1)  
print(cov1.shape)
#128 kernels of size 31 and strides of 4
cov2=(Conv2D(128, 31, strides=4,padding='same'))(cov1)
cov2=(PReLU())(cov2)
print(cov2.shape)
#256 kernels of size 31 and strides of 4
cov3=(Conv2D(256, 31, strides=4,padding='same'))(cov2)
cov3=(PReLU())(cov3)
print(cov3.shape)


(?, 32, 1, 64)
(?, 8, 1, 128)
(?, 2, 1, 256)


## Decoder is starting here

In [29]:
cov4=(Conv2DTranspose(256,31, strides=(1,1),padding='same'))(cov3)
cov4=(PReLU())(cov4)
z1 = merge([cov3,cov4], mode='sum')
cov5=(Conv2DTranspose(128,31, strides=(4,1),padding='same'))(z1)
cov5=(PReLU())(cov5)
z2=merge([cov2,cov5], mode='sum')
cov6=(Conv2DTranspose(64,31, strides=(4,1),padding='same'))(z2)
cov6=(PReLU())(cov6)
z3=merge([cov1,cov6], mode='sum')
cov7=(Conv2DTranspose(16,31, strides=(4,1),padding='same'))(z3)
cov7=(PReLU())(cov7)
cov8=(Conv2DTranspose(1,31, strides=(1,1),activation='tanh',padding='same'))(cov7)
#cov8=(PReLU())(cov8)
cov8=(Reshape((inputSize,1)))(cov8)


C:\Users\Owner\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Owner\Anaconda3\envs\tf\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
C:\Users\Owner\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:6: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
C:\Users\Owner\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:9: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.l

In [30]:
G = Model([inputs,inputs1],output = cov8)


C:\Users\Owner\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model([<tf.Tenso..., outputs=Tensor("re...)`
  """Entry point for launching an IPython kernel.


In [31]:
G.compile(loss=G_loss(cov8,inputs1),optimizer=optim)


In [32]:
G.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 128, 1, 1)     0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 32, 1, 64)     61568       input_1[0][0]                    
____________________________________________________________________________________________________
p_re_lu_1 (PReLU)                (None, 32, 1, 64)     2048        conv2d_1[0][0]                   
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 8, 1, 128)     7872640     p_re_lu_1[0][0]                  
___________________________________________________________________________________________

In [33]:
#discriminator model
inputs=Input((inputSize,1))
# encoder
#model.add(Reshape((16384,1,1),input_shape=input_shape2))
d1=(Conv1D(64, 31, strides=4,padding='same'))
d_hidden1=d1(inputs)
d2=(BatchNormalization())
d_hidden2=d2(d_hidden1)
d3=(LeakyReLU(alpha=0.3))
d_hidden3=d3(d_hidden2)
d4=(Conv1D(128, 31, strides=4,padding='same'))
d_hidden4=d4(d_hidden3)
d5=(BatchNormalization())
d_hidden5=d5(d_hidden4)
d6=(LeakyReLU(alpha=0.3))
d_hidden6=d6(d_hidden5)
d7=(Conv1D(256, 31, strides=4,padding='same'))
d_hidden7=d7(d_hidden6)
d8=(BatchNormalization())
d_hidden8=d8(d_hidden7)
d9=(LeakyReLU(alpha=0.3))
d_hidden9=d9(d_hidden8)
d10=(Conv1D(1, 31, strides=1,padding='same'))
d_hidden10=d10(d_hidden9)
d11=(BatchNormalization())
d_hidden11=d11(d_hidden10)
d12=(LeakyReLU(alpha=0.3))
d_hidden12=d12(d_hidden11)
d13=(Flatten())
d_hidden13=d13(d_hidden12)
d14=Dense(16,activation='sigmoid')   
d_output =d14(d_hidden13)

D= Model(input = inputs,output=d_output)
D.compile(loss=D_loss, optimizer=optim)
#D.compile(loss='mse', optimizer=optim)
D.summary()
# --------------------GAN Model---------------

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 128, 1)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 32, 64)            2048      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 64)            256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 8, 128)            254080    
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 128)            512       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 128)            0         
__________

C:\Users\Owner\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:34: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


# GAN-nam style :)

In [144]:
make_trainable(D,False)
inputs=Input(shape = (inputSize,1,1))
inputs1=Input(shape = (inputSize,1))
input=([inputs,inputs1])
g_output = G(input)
gan_hidden = d1(g_output)
gan_hidden = d2(gan_hidden)
gan_hidden = d3(gan_hidden)
gan_hidden = d4(gan_hidden)
gan_hidden = d5(gan_hidden)
gan_hidden = d6(gan_hidden)
gan_hidden = d7(gan_hidden)
gan_hidden = d8(gan_hidden)
gan_hidden = d9(gan_hidden)
gan_hidden = d10(gan_hidden)
gan_hidden = d11(gan_hidden)
gan_hidden = d12(gan_hidden)
gan_hidden = d13(gan_hidden)
gan_output = d14(gan_hidden)

GAN =Model([inputs,inputs1],output=gan_output)
GAN.compile(loss=GAN_loss(g_output,inputs1), optimizer=optim) 
GAN.summary()
# # --------------------load data--------------------
# c=io.loadmat('feature_all.'
# d=io.loadmat('noise')
clean = cleanSignal
noisy =noisySignal
#noisy=clean+1.5*noise
# # --------------------Main Code--------------------
batch_size=32
n_epochs = 100
n_minibatches = int(noisy.shape[0]/batch_size)

#----------------------  load data ------------


for i in range(n_epochs):
        print ('Epoch:', i+1)
        for index in range(n_minibatches):
            noisy_batch = noisy[index*batch_size:(index+1)*batch_size]
            real_batch = clean[index*batch_size:(index+1)*batch_size]
            
            noisy_batch=np.reshape(noisy_batch,(batch_size,inputSize,1,1))
            real_batch=np.reshape(real_batch,(batch_size,inputSize,1))
            
            combined_G_batch=([noisy_batch,real_batch])
            
            fake_batch = G.predict(combined_G_batch)
            
            fake_batch=np.reshape(fake_batch,(batch_size,inputSize))
            real_batch=np.reshape(real_batch,(batch_size,inputSize))
            print('--------------------enhanced speech Generated!--------------')
            combined_X_batch = np.concatenate((real_batch, fake_batch))
            one_label=np.ones([batch_size, 16])
            zero_label=np.zeros([batch_size, 16])
            combined_y_batch =np.vstack((one_label,zero_label))

            make_trainable(D,True)
            combined_X_batch=np.reshape(combined_X_batch,(2*batch_size,inputSize,1))
            d_loss = D.train_on_batch(combined_X_batch, combined_y_batch)
            print('--------------------Discriminator trained!------------------')
            print(d_loss)
            
            make_trainable(D,False)
            g_loss = GAN.train_on_batch(combined_G_batch,one_label)
            print('--------------------GAN trained!----------------------------')
            print(g_loss)
            

C:\Users\Owner\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model([<tf.Tenso..., outputs=Tensor("de...)`


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_6 (InputLayer)             (None, 128, 1, 1)     0                                            
____________________________________________________________________________________________________
input_7 (InputLayer)             (None, 128, 1)        0                                            
____________________________________________________________________________________________________
model_1 (Model)                  (None, 128, 1)        142776289   input_6[0][0]                    
                                                                   input_7[0][0]                    
____________________________________________________________________________________________________
conv1d_1 (Conv1D)                (None, 32, 64)        2048        model_1[2][0]           

--------------------GAN trained!----------------------------
3.50789
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.115363
--------------------GAN trained!----------------------------
2.26994
Epoch: 12
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.115898
--------------------GAN trained!----------------------------
3.45659
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.11625
--------------------GAN trained!----------------------------
2.24612
Epoch: 13
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.118146
--------------------GAN trained!----------------------------
3.47282
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!---------

--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.115111
--------------------GAN trained!----------------------------
2.06317
Epoch: 32
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.114893
--------------------GAN trained!----------------------------
3.02079
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.114485
--------------------GAN trained!----------------------------
2.02383
Epoch: 33
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.115237
--------------------GAN trained!----------------------------
3.03868
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.116707
--------------------GAN trained!-----------------

--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.112699
--------------------GAN trained!----------------------------
1.81694
Epoch: 52
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.113834
--------------------GAN trained!----------------------------
2.72941
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.113351
--------------------GAN trained!----------------------------
2.4048
Epoch: 53
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.115293
--------------------GAN trained!----------------------------
2.78206
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.11305
--------------------GAN trained!-------------------

--------------------Discriminator trained!------------------
0.111331
--------------------GAN trained!----------------------------
1.6647
Epoch: 72
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.11235
--------------------GAN trained!----------------------------
2.33568
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.11202
--------------------GAN trained!----------------------------
1.74125
Epoch: 73
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.111984
--------------------GAN trained!----------------------------
2.34055
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.111895
--------------------GAN trained!----------------------------
1.84351
Epoch: 74
--------------------enhanced speec

--------------------GAN trained!----------------------------
1.58857
Epoch: 92
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.113016
--------------------GAN trained!----------------------------
2.16137
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.110314
--------------------GAN trained!----------------------------
1.90414
Epoch: 93
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.110659
--------------------GAN trained!----------------------------
2.11223
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.110713
--------------------GAN trained!----------------------------
1.52496
Epoch: 94
--------------------enhanced speech Generated!--------------
--------------------Discriminator traine

--------------------GAN trained!----------------------------
1.5471
Epoch: 112
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.110058
--------------------GAN trained!----------------------------
1.77241
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.111748
--------------------GAN trained!----------------------------
1.42253
Epoch: 113
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.112348
--------------------GAN trained!----------------------------
1.86287
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.112054
--------------------GAN trained!----------------------------
1.84188
Epoch: 114
--------------------enhanced speech Generated!--------------
--------------------Discriminator trai

--------------------GAN trained!----------------------------
1.56436
Epoch: 132
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.11138
--------------------GAN trained!----------------------------
1.58731
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.109775
--------------------GAN trained!----------------------------
1.34175
Epoch: 133
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.110889
--------------------GAN trained!----------------------------
1.60446
--------------------enhanced speech Generated!--------------
--------------------Discriminator trained!------------------
0.111014
--------------------GAN trained!----------------------------
1.56044
Epoch: 134
--------------------enhanced speech Generated!--------------
--------------------Discriminator trai

KeyboardInterrupt: 

# Sanity check in the values : by comparing the difference between fake and real vs noise and real

In [173]:

fake_batch = G.predict(combined_G_batch)

In [174]:
a = np.sum(abs(fake_batch[3] - real_batch[3]))

In [175]:

b = np.sum(abs(noise[0:128]))

In [176]:
print(a/batch_size)

349.648


In [177]:
print(b)

4.20813


In [ ]:
# difference between a and b is small and a is doing good

# Testing time

In [214]:
 y, sr = librosa.load("ToyDataset\\blues.00000.au", duration=8.0)

In [215]:
len(y)

176400

In [216]:
i = 0
step = 128
inp = []
y2 = []
b = noise[128:256]
b = np.array(b)
u = int(len(y)/128)
while i<=u-1:
    inp.append(y[i*step: i*step +step])
    c = np.array(inp[i])
    #c = np.array(inp[i]) + b
    y2.append(c + b)
    i+=1
print(len(c))
len(c.shape)

128


1

In [217]:
print(np.array(y2).shape)

(1378, 128)


In [218]:

#fake_batch = G.predict(combined_G_batch)

y2=np.reshape(np.array(y2),(1378,inputSize,1,1))
inp=np.reshape(np.array(inp),(1378,inputSize,1))

x=([y2,inp])

fake_batch = G.predict(x)

In [219]:
print(fake_batch.shape)

(1378, 128, 1)


In [220]:


noise_test = np.concatenate(y2, axis = 0)

# for y in y2:
#      np.hstack((noise_test, y)
noise_test.shape
# librosa.output.write_wav('file_trim_2s.wav', y2, sr)

(176384, 1, 1)

In [221]:
truth_Test = np.concatenate(fake_batch, axis = 0)

In [222]:
truth_Test.shape

(176384, 1)

In [223]:
test_n=np.reshape(np.array(noise_test),(176384,1))
test_t=np.reshape(np.array(truth_Test),(176384,1))

In [224]:
librosa.output.write_wav('DiffNoiseGanBefore.wav', test_n , sr)
librosa.output.write_wav('DiffNoiseGanAfter.wav', test_t , sr)

# Print noise for future

In [194]:

librosa.output.write_wav("noise.wav",noise,sr)

In [208]:
ynoise, srnoise = librosa.load("noise.wav", duration=3.0)

# split the dataset into parts of 128

In [228]:
def genFilename(filename, part):
    files = filename.split('.')
    return files[0] + '_part_' +str(part) + '_'+ '.' + files[1] + "." + "wav"

def splitFile(filename):
    y, sr = librosa.load("musicDataset//" + filename, duration=10.0)
    i = 0
    step = 128
    b = noise[0:step]
    b = np.array(b)
    u = int(len(y)/step)
    while i<=u-1:
        a = y[i*step: i*step +step]
        librosa.output.write_wav("splitMusicDataset//" + genFilename(filename, i), a, sr)
        c = np.array(a) + b
        librosa.output.write_wav("splitMusicNoiseDataset//" + genFilename(filename, i), b, sr)
        i+=1
    print(len(c))
    len(c.shape)

In [229]:
for filename in os.listdir("musicDataset"):
    splitFile(filename)

128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128


# Skimmed dataset creation

In [ ]:
i = 0
import os
for filename in os.listdir("musicDataset"):
     y, sr = librosa.load("musicDataset//" + filename, duration=10.0)
     files = filename.split('.')
     newfilename = files[0] + '_skim' + '.' + files[1] + "." + "wav"
     noisefilename = files[0] + '_skim' + '.' + files[1] + "." + "wav"
     y1 = y + noise
     #print(newfilename)
#      librosa.output.write_wav("musicDatasetSkim//" + newfilename, y, sr)
     librosa.output.write_wav("musicDatasetNoise//" + noisefilename, y1, sr)

In [264]:
x = np.array(noise[0:32768])
b = np.split(x,len(x)/128)#int(x.shape[0]/2))
print(len(b))
print(len(b[0]))
print(len(b[1]))

256
128
128


In [269]:
c = d
d = b+c
print(len(d))

768


##### 